<a href="https://colab.research.google.com/github/michaelwnau/ai_academy_notebooks/blob/main/ussf_photos_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import requests
from bs4 import BeautifulSoup
from google.colab import drive
import re

#drive.mount('/content/drive')

def download_image(image_url, target_path):
    response = requests.get(image_url, stream=True)
    if response.status_code == 200:
        with open(target_path, 'wb') as f:
            f.write(response.content)

def scrape_images(url, page=0):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    figures = soup.find_all('figure', {'class': 'gallery_container'})

    target_folder = '/content/drive/MyDrive/sf-photos-beautifulsoup'
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    for i, figure in enumerate(figures):
        image_div = figure.find('div', {'class': 'gallery-image'})
        style = image_div['style']
        image_url = re.findall('url\((.*?)\)', style)[0]  # Extract URL from the 'style' attribute
        target_path = os.path.join(target_folder, f'image_{page}_{i}.jpg')
        download_image(image_url, target_path)

    # Check for a 'next page' link and if it exists, follow it
    next_page = soup.find('a', {'class': 'next'})
    if next_page and 'href' in next_page.attrs:
        next_page_url = next_page['href']
        if not next_page_url.startswith('http'):
            next_page_url = url + next_page_url
        scrape_images(next_page_url, page + 1)

scrape_images('https://www.spaceforce.mil/Multimedia/Photos/')
